In [0]:
import tensorflow as tf

# 예제 7.32 : 조선왕조실록 데이터 파일 다운로드
path_to_file = tf.keras.utils.get_file('input.txt', 'http://bit.ly/2Mc3SOV')

In [13]:
# 예제 7.33 : 데이터 로드 및 확인
# 데이터를 메모리에 불러옵니다. 인코딩 형식으로 utf-8을 지정해야 합니다.
train_text = open(path_to_file, 'rb').read().decode(encoding = 'utf-8')

# 텍스트가 총 몇 자인지 확인합니다.
print('Length of text : {} characters'.format(len(train_text)))
print()

# 처음 100자를 확인해봅니다.
print(train_text[: 100])

Length of text : 26265493 characters

﻿태조 이성계 선대의 가계. 목조 이안사가 전주에서 삼척·의주를 거쳐 알동에 정착하다 
태조 강헌 지인 계운 성문 신무 대왕(太祖康獻至仁啓運聖文神武大王)의 성은 이씨(李氏)요, 휘


In [14]:
# 에제 7.34 : 훈련 데이터 입력 정제
import re
# From https://github.com/yoonkim/CNN_sentence/blob/master/process.data.py
def clean_str(string):
  string = re.sub(r"[^가-힣A-Za-z0-9(),!?\'\`]", " ", string)     
  string = re.sub(r"\'ll", " \'ll", string) 
  string = re.sub(r",", " , ", string) 
  string = re.sub(r"!", " ! ", string) 
  string = re.sub(r"\(", " \( ", string) 
  string = re.sub(r"\)", " \) ", string) 
  string = re.sub(r"\?", " \? ", string) 
  string = re.sub(r"\s{2,}", " ", string)
  string = re.sub(r"\'{2,}", "\'", string)
  string = re.sub(r"\'", "", string)

  return string

train_text = train_text.split('\n')
train_text = [clean_str(sentence) for sentence in train_text]
train_text_X = []

for sentence in train_text:
  train_text_X.extend(sentence.split(' '))
  train_text_X.append('\n')

train_text_X = [word for word in train_text_X if word != '']

print(train_text_X[: 20])

['태조', '이성계', '선대의', '가계', '목조', '이안사가', '전주에서', '삼척', '의주를', '거쳐', '알동에', '정착하다', '\n', '태조', '강헌', '지인', '계운', '성문', '신무', '대왕']


In [15]:
# 예제 7.35 : 단어 토큰화

import numpy as np

# 단어의 set을 만듭니다.
vocab = sorted(set(train_text_X))
vocab.append('UNK')
print('{} unique words'.format(len(vocab)))

# vocab list를 숫자로 매핑하고, 반대도 실행합니다.
word2idx = {u:i for i, u in enumerate(vocab)}
idx2word = np.array(vocab)

text_as_int = np.array([word2idx[c] for c in train_text_X])

# word2idx 의 일부를 알아보기 쉽게 출력해 봅니다.
print('{')
for word, _ in zip(word2idx, range(10)):
  print(' {:4s}: {:3d},'.format(repr(word), word2idx[word]))
print(' ...\n}')

print('index of UNK: {}'.format(word2idx['UNK']))

332515 unique words
{
 '\n':   0,
 '!' :   1,
 ',' :   2,
 '000명으로':   3,
 '001':   4,
 '002':   5,
 '003':   6,
 '004':   7,
 '005':   8,
 '006':   9,
 ...
}
index of UNK: 332514


In [16]:
# 예제 7.36 : 토큰 데이터 확인
print(train_text_X[: 20])
print(text_as_int[: 20])

['태조', '이성계', '선대의', '가계', '목조', '이안사가', '전주에서', '삼척', '의주를', '거쳐', '알동에', '정착하다', '\n', '태조', '강헌', '지인', '계운', '성문', '신무', '대왕']
[299181 229520 161349  17366 110944 230178 250960 154993 225348  28960
 190192 256005      0 299181  25557 273428  36069 163902 180371  84330]


In [17]:
# 예제 7.37 : 기본 데이터세트 만들기
seq_length = 25
examples_per_epoch = len(text_as_int) // seq_length
sentence_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sentence_dataset = sentence_dataset.batch(seq_length + 1, drop_remainder = True)

for item in sentence_dataset.take(1):
  print(idx2word[item.numpy()])
  print(item.numpy())

['태조' '이성계' '선대의' '가계' '목조' '이안사가' '전주에서' '삼척' '의주를' '거쳐' '알동에' '정착하다'
 '\n' '태조' '강헌' '지인' '계운' '성문' '신무' '대왕' '\\(' '\\)' '의' '성은' '이씨' '\\(']
[299181 229520 161349  17366 110944 230178 250960 154993 225348  28960
 190192 256005      0 299181  25557 273428  36069 163902 180371  84330
  17278  17279 224068 164455 230134  17278]


In [18]:
# 예제 7.38 : 학습 데이터세트 만들기
def split_input_target(chunk):
  return [chunk[: -1], chunk[-1]]

train_dataset = sentence_dataset.map(split_input_target)
for x, y in train_dataset.take(1):
  print(idx2word[x.numpy()])
  print(x.numpy())
  print(idx2word[y.numpy()])
  print(y.numpy())

['태조' '이성계' '선대의' '가계' '목조' '이안사가' '전주에서' '삼척' '의주를' '거쳐' '알동에' '정착하다'
 '\n' '태조' '강헌' '지인' '계운' '성문' '신무' '대왕' '\\(' '\\)' '의' '성은' '이씨']
[299181 229520 161349  17366 110944 230178 250960 154993 225348  28960
 190192 256005      0 299181  25557 273428  36069 163902 180371  84330
  17278  17279 224068 164455 230134]
\(
17278


In [0]:
# 예제 7.39 : 데이터세트 shuffle, batch 설정
BATCH_SIZE = 128
steps_per_epoch = examples_per_epoch // BATCH_SIZE
BUFFER_SIZE = 10000

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

In [20]:
# 예제 7.40 : 단어 단위 생성 모델 정의
total_words = len(vocab)
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(total_words, 100, input_length = seq_length),
                             tf.keras.layers.LSTM(units = 100, return_sequences = True),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.LSTM(units = 100),
                             tf.keras.layers.Dense(total_words, activation = 'softmax')
])

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 100)           33251500  
_________________________________________________________________
lstm_2 (LSTM)                (None, 25, 100)           80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 25, 100)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 332515)            33584015  
Total params: 66,996,315
Trainable params: 66,996,315
Non-trainable params: 0
_________________________________________________________________


In [0]:
# 예제 7.41 : 단어 단위 생성 모델 학습
from tensorflow.keras.preprocessing.sequence import pad_sequences
def testmodel(epoch, logs):
  if epoch % 5 != 0 and epoch != 49:
    return
  test_sentence = train_text[0]

  next_words = 100
  for _ in range(next_words):
    test_text_X = test_sentence.split(' ')[-seq_length :]
    test_text_X = np.array([word2idx[c] if c in word2idx else word2idx['UNK'] for c in test_text_X])
    test_text_X = pad_sequences([test_text_X], maxlen = seq_length, padding = 'pre', value = word2idx['UNK'])

    output_idx = model.predict_classes(test_text_X)
    test_sentence += ' '+ idx2word[output_idx[0]]

  print()
  print(test_sentence)
  print()

testmodelcb = tf.keras.callbacks.LambdaCallback(on_epoch_end = testmodel)

history = model.fit(train_dataset.repeat(), epochs = 50, steps_per_epoch = steps_per_epoch, callbacks = [testmodelcb])

Epoch 1/50
2184/2719 [=======================>......] - ETA: 14:40 - loss: 7.7264 - accuracy: 0.1865

In [0]:
# 예제 7.42 : 임의의 문장을 사용한 생성 결과 확인
from tensorflow.keras.preprocessing import pad_sequences
test_sentence = '동헌에 나가 공무를 본 후 활 십오 순을 쏘았다'

next_words = 100
for _ in range(next_words):
  test_text_X = test_sentence.split(' ')[-seq_length :]
  test_text_X = np.array([word2idx[c] if c in word2idx else total_words + 1 for c in test_text_X])
  test_text_X = pad_sequences([test_text_X], maxlen = seq_length, padding = 'pre', value = word2idx['UNK'])
  
  output_idx = model.predict_classes(test_text_X)
  test_sentence += ' ' + idx2word[output_idx[0]]

print(test_sentence)